In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')  

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Check GPU connection
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Sep  9 08:14:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Check RAM access
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


<a href="https://colab.research.google.com/github/iakioh/MusiCAN/blob/main/models/first_music_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# musiGAN

**Description:** 1-Track MuseGAN architecture build on MiniGAN.\
**Purpose:** implement a composing GAN.\
**Results:**

In [4]:
import pickle
from tqdm import notebook

import math
import numpy as np
import torch
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

## Class and function definitions

### Data creation

In [5]:
class CentralDotImage :
    """
        creates mock data set out of a simple image copied multiple times

        Methods
        -------
        __init__(height, width, dataset_size = 100) : 
            creates all attributes
        show() : 
            plt.plots image

        Attributes
        ----------
        height : int
            of image, in pixels
        width : int
            of image, in pixels
        size : int
            number of pixels in image
        dataset_size : int
            number of images
        
        image : torch.Tensor
            out of 1s and 0s,
            size = (width, height)
        vector : torch.Tensor
            flattened image,
            size = (size)
        data : torch.Tensor
            images copied dataset_size times
            size = (dataset_size, size)
        dataset : torch.TensorDataset
    """

    def __init__ (self, height, width, dataset_size = 100) :
        # Input checks
        assert  type(height) == int 
        assert  height >= 1 
        assert  type(width) == int
        assert  width >= 1
        assert  type(dataset_size) == int
        assert  dataset_size >= 1

        self.height = height
        self.width  = width
        self.shape  = (height, width)
        self.size   = height * width
        self.dataset_size = dataset_size

        # Image creation
        self.image = torch.zeros(height, width)
        for i in range(height) :
            for j in range(width) :
                self.image[i, j] = \
                    i + 1 <= math.ceil(3/4 * height) and \
                    i + 1 > math.floor(1/4 * height) and \
                    j + 1 <= math.ceil(3/4 * width) and \
                    j + 1 > math.floor(1/4 * width)

        # Dataset creation
        self.vector  = self.image.flatten()
        self.data    = self.vector[None, :].expand(dataset_size, self.size)
        self.dataset = torch.utils.data.TensorDataset(self.data)

    def show (self) :
        plt.imshow(self.image)
        plt.show()

In [6]:
class Pianoroll :
    def __init__ (self, filepath) :
        assert  type(filepath) == str

        # Creating the dataset from a file
        stored_data = np.load(filepath)
        data_array  = stored_data["data"]
        label_array = stored_data["labels"]
        self.data   = torch.as_tensor(data_array, dtype = torch.float32)
        self.labels = torch.as_tensor(label_array, dtype = torch.int64)

        self.dataset = torch.utils.data.TensorDataset(self.data, self.labels)

        # Storing additional info about it
        self.shape  = tuple(self.data.shape[1:])   # shape of one pianoroll image
        self.size   = self.shape[0] * self.shape[1]
        self.height       = self.data.shape[2]
        self.width        = self.data.shape[1]
        self.dataset_size = self.data.shape[0]

        self.genre_list = ['Rap', 'Latin', 'International', 'Electronic', 
                           'Country', 'Folk', 'Blues', 'Reggae', 'Jazz',
                           'Vocal', 'New-Age', 'RnB', 'Pop_Rock']
    
    def show (self, number = None) :
        if number == None :
            number = np.random.randint(self.dataset_size)
        else :
            assert  type(number) == int
            assert  number >= 0 and number < self.dataset_size

        plt.figure(figsize = (12, 3))
        plt.title(f"pianoroll #{number}")
        plt.imshow(self.data[number].T)
        plt.show()    

### Architecture

#### Support classes

In [7]:
"""
    These two classes serves as torch layers to binarize the output of the Generator while keeping the layer still "backpropagatable" (via a hardtanh).
    This is not our own code. For source, see:
    https://www.hassanaskary.com/python/pytorch/deep%20learning/2020/09/19/intuitive-explanation-of-straight-through-estimators.html#:~:text=A%20straight%2Dthrough%20estimator%20is,function%20was%20an%20identity%20function.
"""

class STEFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return (input > 0).float()

    @staticmethod
    def backward(ctx, grad_output):
        return torch.nn.functional.hardtanh(grad_output)

class StraightThroughEstimator(torch.nn.Module):
    def __init__(self):
        super(StraightThroughEstimator, self).__init__()

    def forward(self, x):
        x = STEFunction.apply(x)
        return x


#### Parent Network classes

In [8]:
class Network (torch.nn.Module) :
    """
        General neural network class for specialized networks.
        Create those by:
        ```
        myNet = Network(<int_input_length>, <int_output_length>, <str_name>)
        layer_list = [<torch.nn.Module1>, ...]
        myNet.create_model(layer_list)
        myNet.print_stats()
        ```
    """


    def __init__ (self) : 
        super().__init__()
    

    def print_stats (self) :
        print(f"{self.name}:")
        print(f"    input length:  {self.I}")
        print(f"    output length: {self.O}")
        if hasattr(self, 'model') :
            print(f"    layers:        {len(self.model)}")
            print(f"    parameters:    {self.count_params()}")
        else :
            print(f"    Create the network architecture with .create_model(<layer_list>)")
        print("")


    def count_params (self) :
        """count number of trainable parameters"""

        return sum(p.numel() for p in self.parameters() if p.requires_grad)


    def forward (self, input, numpy_out = False) :
        """run network"""
        output = self.model(input)
        if numpy_out :
            output = self._numpify(output)

        return output
    
    
    def _numpify (self, tensor) :
        return tensor.cpu().detach().numpy()

In [9]:
class Generator (Network) :
    """
        GAN Generator class whose architecture is custom defined after initialization:
        ```
        myGen = Generator(<int_input_length>, <int_output_length>, <str_name>)
        layer_list = [<torch.nn.Module1>, ...]
        myGen.create_model(layer_list)
        myGen.print_stats()
        ```

        There are two forward() variants:
        * `forward(batchsize = 0, numpy_out = False)` with auto-generated normally distributed seeds.
        * `forward_custom(seed, numpy_out = False)` with custom seeds as input.
    """

    def forward (self, batch_size = 0, numpy_out = False) :
        """run network with a batch of normally distributed seeds"""
        assert  type(batch_size) == int
        assert  batch_size >= 0

        # Seed generation
        if batch_size == 0 :
            seed_size = (self.I,)
        else :
            seed_size = (batch_size, self.I)
        seed = torch.normal(0, 1, seed_size)
        
        # Running the network
        return super().forward(seed, numpy_out)

            
    def forward_custom (self, seed, numpy_out = False) :
        """run network with custom seed"""
        assert  type(seed) == torch.Tensor
        assert  seed.shape[-1] == self.I

        # Running the network
        return super().forward(seed, numpy_out = numpy_out)



In [10]:
class Discriminator (Network) :
    """
        GAN Discriminator class whose architecture is custom defined after initialization:
        ```
        myDis = Discriminator(<int_input_length>, <int_output_length>, <str_name>)
        layer_list = [<torch.nn.Module1>, ...]
        myDis.create_model(layer_list)
        myDis.print_stats()
        ```
    """

    def forward (self, input, prob_out = False, numpy_out = False) :
        """
            run network, optionally return probability (default logit) or numpy array (default torch)
        """
        assert  type(input) in [torch.Tensor, np.ndarray]
        assert  input.shape[-1] == self.I

        if type(input) == np.ndarray :
            input = torch.tensor(input)

        # Running the network
        
        output = super().forward(input)
        if prob_out :
            output = torch.sigmoid(output)
        if numpy_out :
            output = self._numpify(output)

        return output

### Training

#### Support classes

In [11]:
def unif_cross_entropy(probabilities, weight):
  return(torch.mean(weight * torch.log(probabilities)))

In [12]:
def generator_goodness (generated_batch, real_batch) :
    """
        compare two batches of data by calculating the absolute mean difference
    """
    
    # averaged over batches 
    generated_mean = torch.mean(generated_batch)
    real_mean      = torch.mean(real_batch)

    # take differnece & absolut value, average over features lastly
    goodness_criteria = torch.mean(torch.abs(real_mean - generated_mean))

    return goodness_criteria.cpu().detach().numpy()

In [13]:
class Log :
    """
        container class for GANTraining logs
    """
    
    def __init__ (self) :
        self.losses       = None
        self.music_probs  = None # probs how real / fake music
        self.genre_probs  = None # probs of each genre labels of music
        self.gen_goodness = None
        self._dis_losses  = None
        self._music_probs = None
        self._genre_probs = None

In [14]:
def quick_test(generator, discriminator, data, test_size = 10, num_images = 1): 
    
    # Test on generated data
    generator.eval()
    discriminator.eval()
    device = 'cuda'  if torch.cuda.is_available() else  'cpu'
    data_real = iter(torch.utils.data.DataLoader(data.dataset,  # make a batch
                     batch_size = test_size, 
                     shuffle = True, drop_last = True)
                ).next()[0].to(device)
    
    ## Generator
    data_generated = generator.forward(batch_size = test_size)

    ## Discriminator

    gen_judgements  = discriminator.forward(data_generated)
    real_judgements = discriminator.forward(data_real)
    gen_probs       = torch.sigmoid(gen_judgements)
    real_probs      = torch.sigmoid(real_judgements)
    gen_prob        = torch.std_mean(gen_probs, unbiased=True)
    real_prob       = torch.std_mean(real_probs, unbiased=True)
    
    # Discriminator Results
    print(f"Discriminator p(x_real = real) = " +
          f"{real_prob[1]*100:.0f}±{real_prob[0]*100:.0f}%")
    print(f"Discriminator p(x_gen = real)  = " +
          f"{gen_prob[1]*100:.0f}±{gen_prob[0]*100:.0f}%")
    
    # Generator examples
    images  = data_generated.cpu().detach().numpy() \
              .reshape(-1, data.height, data.width)
    np.random.shuffle(images)  # shuffles ordering of images 
    
    plt.title("Generated examples")
    for i in range(num_images) :
        plt.figure(figsize = (12, 3))
        plt.imshow(images[i])
        plt.show()

#### GANTraining

In [15]:
from IPython.core.display import Path
class GANTraining :
    """
        general GAN training class
        How To Use:
        * `MyTrain = GANTraining(<Generator>, <Discriminator>, <torch_dataset>)`
        * `MyTrain.setup(<int_rounds>, batchsize = 1, discriminator_rounds = 1,     
                        loss_function = ["WGAN", "GAN"])`
        * `MyTrain.train()`
      
        After That:
        * `MyTrain.gen` contains trained Generator
        * `MyTrain.dis` contains trained Discriminator
        * `MyTrain.log` contains metrics from each round:
            - .losses       : np.array, shape = (5, rounds)
            - .probs        : np.array, shape = (2, rounds)
            - .gen_goodness : np.array, shape = (rounds,)
    """


    def __init__ (self, Gen, Dis, dataset) :
        # Input checks
        #assert  issubclass(Gen, Generator)
        #assert  issubclass(Dis, Discriminator)
        assert  type(dataset) == torch.utils.data.dataset.TensorDataset
        #assert  Gen().O == Dis().I
        #assert  Dis().O == 1
        #assert  dataset.tensors[0].shape[-1] == Gen().O   # real and gen. data should have the same size

        self.device = 'cuda'  if torch.cuda.is_available() else  'cpu'

        # GAN classes and dataset
        self.n_labels = 13     # number of labels in dataset, automate maybe
        self.GenClass = Gen
        self.DisClass = Dis
        self.dataset  = dataset
        #self.dataset.tensors[0].to(self.device)
        

    def setup (self, rounds, batch_size = 1, discriminator_rounds = 1, 
               loss_function = "CAN") :
        assert  type(rounds) == int
        assert  rounds >= 1
        assert  type(batch_size) == int
        assert  batch_size >= 1
        assert  type(discriminator_rounds) == int
        assert  discriminator_rounds >= 1
        assert  loss_function in ["GAN", "WGAN", "CAN"]

        # Training parameters
        self.rounds     = rounds
        self.batch_size = batch_size
        self.dis_rounds = discriminator_rounds
        self.loss       = loss_function

        # Dataloader
        self.data_loader = torch.utils.data.DataLoader(self.dataset,
                                batch_size = self.batch_size, 
                                drop_last = True,
                                shuffle = True)
        self.dataset_size = self.dataset.tensors[0].shape[0]  # number of instances in dataset
        self.batch_count = self.dataset_size // self.batch_size
        self._batch_idx  = self.batch_count 
        
        # Logs
        self.log = Log()
        self.log.losses       = np.zeros((5, self.rounds)) 
        self.log.music_probs  = np.zeros((2, self.rounds))
        self.log.genre_probs = np.zeros((1 + self.n_labels, self.rounds))
        self.log.gen_goodness = np.zeros((self.rounds,))
        self.log._dis_losses  = torch.zeros((4, self.dis_rounds)).cpu()
        self.log._music_probs = torch.zeros((2, self.dis_rounds)).cpu()
        self.log._genre_probs = torch.zeros((1 + self.n_labels, self.rounds)).cpu()

    def train (self) :
        assert  hasattr(self, "data_loader")  # If test fails, you haven't run set_params()

        print(f"Training")
        self._initialize_GAN()
        arranged_tensor = torch.arange(self.batch_size) # used each round

        for round in notebook.tqdm(range(self.rounds)) :
            for dis_round in range(self.dis_rounds) :
                print(f"Dis. round {dis_round + 1}           ", end="\r")  # print one dot for every dis_round
                
                # Forward propagation
                batch_real, labels_real = self._get_batch()
                # print("dis batch", batch_real.device)
                batch_gen  = self.gen.forward(batch_size = self.batch_size)
                print("batch_gen complete     ", end = '\r')
                music_dis_real, genre_dis_real = self.dis.forward(batch_real)
                print("dis_real complete      ", end = '\r')
                music_dis_gen, genre_dis_gen = self.dis.forward(batch_gen)
                print("dis_gen complete       ", end = '\r')

                self.music_prob_real = torch.sigmoid(music_dis_real)
                genre_probs_real = torch.nn.functional.softmax(genre_dis_real, dim = 1)
                self.genre_prob_real = genre_probs_real[arranged_tensor, labels_real] # get prob of real genre
                self.music_prob_gen  = torch.sigmoid(music_dis_gen)
                self.genre_probs_gen  = torch.nn.functional.softmax(genre_dis_gen, dim = 1)
                
                # Calculating the Discriminator loss function
                if self.loss == "GAN" :
                    self.loss_real = - torch.mean(torch.log(self.music_prob_real))
                    self.loss_gen  = - torch.mean(torch.log(1 - self.music_prob_gen))
                    self.loss_reg  = torch.tensor(0.)
                
                elif self.loss == "WGAN" :
                    var_gen   = torch.var(music_dis_gen)
                    var_real  = torch.var(music_dis_real)
                    self.loss_reg  = torch.where(var_gen > 1, 
                                                 (var_gen - 1)**2, 0) \
                                     + torch.where(var_real > 1, 
                                                   (var_real - 1)**2, 0)
                    self.loss_real = - torch.mean(music_dis_real)
                    self.loss_gen  = torch.mean(music_dis_gen)
                
                elif self.loss == "CAN" :
                    self.loss_real_music = - torch.mean(torch.log(self.music_prob_real))
                    self.loss_real_genre = - torch.mean(torch.log(self.genre_prob_real))
                    self.loss_real = self.loss_real_music + self.loss_real_genre
                    self.loss_gen = - torch.mean(torch.log(1 - self.music_prob_gen))
                    self.loss_reg  = torch.tensor(0.)

                self.loss_dis = self.loss_real + self.loss_gen + self.loss_reg
                self._log_all(round, k = dis_round)
                print("loss_dis complete      ", end = '\r')
                
                # Discriminator update
                self.optimizer_dis.zero_grad()
                self.loss_dis.backward()
                self.optimizer_dis.step()
                print("dis.backward() complete", end = '\r')


            # Calculating the Generator loss function
            batch_new = self.gen.forward(batch_size = self.batch_size)
            print("batch_gen complete     ", end = '\r')
            music_dis_new, genre_dis_new = self.dis.forward(batch_new)
            # print("dis new output:", music_dis_new.size(), genre_dis_new.size())
            print("dis_new complete       ", end = '\r')
                
            if self.loss == "GAN" :
                music_prob_new = torch.sigmoid(music_dis_new)
                self.loss_gen = -torch.mean(torch.log(music_prob_new)) 
            elif self.loss == "WGAN" :
                self.loss_gen = -torch.mean(music_dis_new)
            elif self.loss == "CAN" :
                music_prob_new = torch.sigmoid(music_dis_new)
                self.loss_gen_music = -torch.mean(music_prob_new)
                genre_probs_new = torch.nn.functional.softmax(genre_dis_new, dim = 1)
                # print(genre_probs_new.size())
                self.loss_gen_genre = - torch.mean( \
                    unif_cross_entropy(genre_probs_new, 1 / self.n_labels) + \
                    unif_cross_entropy(1 - genre_probs_new, 1 - 1 / self.n_labels))
                self.loss_gen = self.loss_gen_music + self.loss_gen_genre

            self._log_all(round)
            print("loss_gen complete      ", end = '\r')
                
            
            # Generator update
            self.optimizer_gen.zero_grad()
            self.loss_gen.backward()
            self.optimizer_gen.step()
            print("gen.backward() complete", end = '\r')

        # Put GAN in eval mode
        self.gen.eval()
        self.dis.eval()
        print("Training complete. GAN now in eval() mode.")


    def _initialize_GAN (self) :
        """
            Before each training, the Generator and Discriminator will be freshly initialized with new random weights.
        """
        self.gen = self.GenClass().to(self.device)
        self.dis = self.DisClass().to(self.device)
        self.optimizer_gen = torch.optim.Adam(self.gen.parameters(), 
                                              lr = 0.001,
                                              betas = (0.5, 0.9))
        self.optimizer_dis = torch.optim.Adam(self.dis.parameters(), 
                                              lr = 0.001,
                                              betas = (0.5, 0.9))
        # Note: ADAM parameters from GAN tutorial [1].
        

    def _get_batch (self) :
        """
            samples one batch of data from self.data_loader without replacement.
            When the self.data_set is depleted of fresh batches, 
            self.data_loader will shuffle a list of new batches.
        """
        if self._batch_idx >= self.batch_count :
            self._data_iter = iter(self.data_loader)
            self._batch_idx = 0
        batch_data, batch_labels = self._data_iter.next()
        self._batch_idx += 1

        return batch_data.to(self.device), batch_labels.to(self.device)



    def _log_all (self, round, k = -1) :
        if k >= 0 : # before each Discriminator update
            self.log._dis_losses[0, k] = self.loss_dis.cpu().detach()
            self.log._dis_losses[1, k] = self.loss_real.cpu().detach()
            self.log._dis_losses[2, k] = self.loss_gen.cpu().detach()
            self.log._dis_losses[3, k] = self.loss_reg.cpu().detach()
            self.log._music_probs[0, k] = self.music_prob_real.mean().cpu().detach()
            self.log._music_probs[1, k] = self.music_prob_gen.mean().cpu().detach()
            self.log._genre_probs[0, k]  = self.genre_prob_real.mean().cpu().detach() # prob of right label of real batch
            self.log._genre_probs[1:, k] = self.genre_probs_gen.mean().cpu().detach() # prob of genres of generated batch
        
        if k == -1 : # before each Generator update
            # Losses
            dis_losses = self.log._dis_losses.detach().cpu().numpy()
            self.log.losses[0:4, round] = dis_losses.mean(axis = 1)
            self.log.losses[4, round]   = self.loss_gen.detach().cpu().numpy()
            
            # Discriminator Probabilities
            music_probs                 = self.log._music_probs.cpu().detach().numpy()
            genre_probs                 = self.log._genre_probs.cpu().detach().numpy()
            self.log.music_probs[:, round] = music_probs.mean(axis = 1)
            self.log.genre_probs[:, round] = genre_probs.mean(axis = 1)

            # Generator Goodness
            batch_real, _ = self._get_batch()
            batch_gen  = self.gen.forward(batch_size = self.batch_size)
            goodness   = generator_goodness(batch_gen, batch_real)
            self.log.gen_goodness[round] = goodness


    def plot (self) :
        rounds = np.arange(self.rounds) + 1
        plt.suptitle("Training results")
        plt.title("Loss Terms")
        plt.plot(rounds, self.log.losses.T)
        plt.xlabel("round")
        plt.ylim(-5, 5)
        plt.legend(["Discriminator Loss", "Dis. Real Term", "Dis. Gen Term", "Dis. Reg Term", "Generator Loss"])
        plt.show()

        plt.title("Batch-averaged Discriminator Probabilities during Training")
        plt.plot(rounds, self.log.music_probs.T)
        plt.plot(rounds, self.log.genre_probs[0,:])
        plt.xlabel("epoch")
        plt.ylim(0,1)
        plt.legend(["real data", "generated data", "true label"])
        plt.show()

        plt.title("Batch-averaged Generator Probabilities during Training")
        plt.plot(rounds, self.log.genre_probs[1:,:].T)
        plt.xlabel("epoch")
        plt.ylim(0,1)
        plt.legend(lpd5.genre_list)
        plt.show()

        plt.title("Generator Goodness during Training")
        plt.plot(rounds, self.log.gen_goodness)
        plt.xlabel("epoch")
        plt.ylim(0,1)
        plt.legend(["Averaged real-generated difference"])
        plt.show()



    def save (self, training_filepath, name) :
        file_name = training_filepath + name
        torch.save(self.gen.state_dict(), file_name + "_gen.obj")
        torch.save(self.dis.state_dict(), file_name + "_dis.obj")

        print(f"Saved training under '{file_name}'")

    def load (self, training_filepath, name) :
        file_name = training_filepath + name 
        self.gen.load_state_dict(torch.load(file_name + "_gen.obj"))
        self.dis.load_state_dict(torch.load(file_name + "_dis.obj"))
        
        print(f"Loaded training from '{file_name}'")


# Sources:
# [1] https://github.com/salu133445/ismir2019tutorial/blob/main/gan.ipynb

## Network training and evaluation

**Template for own Generator or Discriminator class**
```python
class SETCLASSNAME (Generator OR Discriminator) :
    def __init__ (self) :
        super().__init__()
        self.name = self.__class__.__name__

        self.I     = #SET SEED LENGTH
        self.O     = #SET OUTPUT LENGTH
        self.model = torch.nn.Sequential(
            # ENTER TORCH.NN MODULES
        )
```

### musiCAN

#### LPD5

In [ ]:
data_name = "lpd5_full_4bars"
lpd5 = Pianoroll("/content/drive/MyDrive/MusiCAN_data_and_models/" + data_name + "/prepared_arrays.npz")

In [ ]:
lpd5.show()

In [ ]:
lpd5.bars = 4
lpd5.blips_per_bar = lpd5.width // lpd5.bars
lpd5.pitches = lpd5.height
lpd5.octaves = lpd5.pitches // 12

print("lpd5.blips_per_bar", lpd5.blips_per_bar)
print("lpd5.pitches", lpd5.pitches)
print("lpd5.octaves", lpd5.octaves)

#### Generator

In [ ]:
class GeneratorBlock(torch.nn.Module):
    """ 2d transconv layer, batch normalization & ReLU """

    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()

        self.gen_block = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(in_dim, out_dim, kernel, stride),
            torch.nn.BatchNorm2d(out_dim),
            torch.nn.ReLU()
            )

    def forward(self, x):
        return self.gen_block(x)

In [ ]:
class MusiGen (torch.nn.Module) :
    """
    1-track museGAN generator, consisting of two sub-networks (so-called 
    temporal and bar generator)

    input : seed vector, a normally distributed random vector, 
            length: (B + 1) * 64 = 5 * 64 here
    output: pianaroll, binary tensor, shape: (B x T x P) = (4 x 48 x 84) here
    """

    def __init__ (self, log = False, **kwargs) : 
        super().__init__(**kwargs)

        # Data parameters
        self.octaves    = lpd5.octaves
        self.bars       = lpd5.bars    # bars per pianoroll
        self.T          = lpd5.blips_per_bar  # timesteps per bar
        self.P          = lpd5.pitches   # pitches
        self.seedlength = 64
        
        self.temporal_generator = torch.nn.Sequential(
            
            # heuristically added linear layer
            torch.nn.Linear(1, 31),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),

            # transconv layer 1
            torch.nn.ConvTranspose1d(64, 1024, 2, 2),
            torch.nn.BatchNorm1d(1024),
            torch.nn.ReLU(),

            # transconv layer 2
            torch.nn.ConvTranspose1d(1024, 1, 3, 1),
            torch.nn.BatchNorm1d(1),
            torch.nn.ReLU()
        )

        self.bar_generator = torch.nn.Sequential(
            
            # transconv layers
            GeneratorBlock( 128, 1024, (2, 1), (2, 1)),
            GeneratorBlock(1024,  512, (2, 1), (2, 1)),
            GeneratorBlock( 512,  256, (2, 1), (2, 1)),
            GeneratorBlock( 256,  256, (2, 1), (2, 1)),
            # GeneratorBlock( 256,  256, (2, 1), (2, 1)),   # added
            GeneratorBlock( 256,  128, (3, 1), (3, 1)),
            GeneratorBlock( 128,   64, (1, self.octaves), (1, self.octaves)),

            # last layer with tanh & binarization activation fct.s
            torch.nn.ConvTranspose2d(64, 1, (1, 12), (1, 12)),
            torch.nn.BatchNorm2d(1),
            torch.nn.Tanh(),
            StraightThroughEstimator() # binarization
        )

        if log :
            print(f"Generator: parameters: {self.count_params()}")
            print("")

    def count_params (self) :
        """count number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


    def forward_custom (self, seed) :
        assert  type(seed) == torch.Tensor
        assert  len(seed.shape) == 2
        assert  seed.shape[0] >= 1
        assert  seed.shape[1] == (1 + self.bars) * self.seedlength

        batchsize = seed.shape[0]
        return self.forward(batchsize, seed)


    def forward (self, batch_size, seed = None) :
        
        if seed == None :
            assert type(batch_size) == int
            assert batch_size >= 1
            device = 'cuda'  if torch.cuda.is_available() else  'cpu'
            seed = torch.normal(0., 1, (batch_size, (1 + self.bars) * self.seedlength)).to(device)
            
        seeds = torch.chunk(seed, chunks = 5, dim = 1)
        
        # create time-independent first half of seed for bar generator
        bar_seed_1 = seeds[0]
        bar_seed_1 = bar_seed_1.view((-1, self.seedlength, 1, 1)) # reshape for transconv layers

        # generate pianorolls bar by bar
        generated_bars = []
        for temporal_seed in seeds[1:]:
            
            ## generate time-dependent second half of seed for bar generator

            temporal_seed = temporal_seed.view(-1, self.seedlength, 1) # reshape for transconv layers
            #print(f"temporal seed: {temporal_seed.size()}")
            bar_seed_2 = self.temporal_generator(temporal_seed) # (batch size x 1 x 64)
            #print(f"bar seed 2: {bar_seed_2.size()}")

            ## reshape & concatenate both halfs of seed for bar generator 
            
            bar_seed_2 = bar_seed_2.view(-1, self.seedlength, 1, 1)
            bar_seed   = torch.cat((bar_seed_1, bar_seed_2), dim = 1) # (batch size x 128 x 1 x 1)
            #print(f"bar seed: {bar_seed d.size()}")

            ## generate one bar 
            
            generated_bar = self.bar_generator(bar_seed) # (batch size x 1 x 24 x 84)
            #print(f"generated_bar: {generated_bar.size()}")
            generated_bars.append(generated_bar) 

        pianoroll = torch.cat(generated_bars, dim = 1) # (batch size x 4 x 24 x 84) 
        #print(f"gen output: {pianoroll.size()}")

        return pianoroll

#### Discriminator

In [ ]:
class DiscriminatorBlock(torch.nn.Module):
    """3d conv layer & Leaky ReLU"""

    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.dis_block = torch.nn.Sequential(
            torch.nn.Conv3d(in_dim, out_dim, kernel, stride),
            torch.nn.LeakyReLU(negative_slope = 0.2)   # MuseGAN Hyperparameter
            )

    def forward(self, x):
        return self.dis_block(x)

In [ ]:
class MusiDis (torch.nn.Module) :
    """
    1-Track musiCAN discriminator, with 2 heads 
    
    input : (B x T x P) binary pianoroll

    output: 1. single number, prob. that the input pianoroll is a 
            real and not generated
            2. vector of length = number of genres, prob. that the input 
            pianoroll is of a certain genre type

    n_labels : number of labels
    """

    def __init__ (self, n_labels = 13, log = False, **kwargs) :
        super().__init__(**kwargs)

        # Data parameters
        self.octaves    = lpd5.octaves
        self.bars       = lpd5.bars    # bars per pianoroll
        self.T          = lpd5.blips_per_bar  # timesteps per bar
        self.P          = lpd5.pitches   # pitches
        self.seedlength = 64
        self.n_labels   = n_labels
      
        # common body: conv layers
        self.discriminator_conv = torch.nn.Sequential(
            DiscriminatorBlock(  1, 128, (2, 1,  1), (1, 1,  1)),
            DiscriminatorBlock(128, 128, (3, 1,  1), (1, 1,  1)),
            DiscriminatorBlock(128, 128, (1, 1, 12), (1, 1, 12)), 
            DiscriminatorBlock(128, 128, (1, 1,  self.octaves), (1, 1,  self.octaves)),
            DiscriminatorBlock(128, 128, (1, 2,  1), (1, 2,  1)),
            DiscriminatorBlock(128, 128, (1, 2,  1), (1, 2,  1)),
            DiscriminatorBlock(128, 256, (1, 4,  1), (1, 2,  1)),
            DiscriminatorBlock(256, 512, (1, 3,  1), (1, 2,  1))
            )
        
        # heads: fully-connected layers
        self.discriminator_music_head = torch.nn.Sequential(
            torch.nn.Linear(512*2, 1024),  
            torch.nn.LeakyReLU(negative_slope = 0.2),
            torch.nn.Linear(1024, 1))
        
        self.discriminator_genre_head = torch.nn.Sequential(
            torch.nn.Linear(512*2, 1024),  
            torch.nn.LeakyReLU(negative_slope = 0.2),
            torch.nn.Linear(1024, self.n_labels))

        if log :
            print(f"Discriminator parameters: {self.count_params()}")
            print("")

    def count_params (self) :
        """count number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def forward (self, pianoroll):

        # reshape input for transconvs
        pianoroll   = pianoroll.view(-1, 1, self.bars, self.T, self.P) 
        # print("dis input prep.", pianoroll.shape)

        # put through common body and flatten instances
        common_conv_output = self.discriminator_conv(pianoroll)
        common_fc_input = common_conv_output.view(-1, 512*2)  
        # print("dis conv out", common_conv_output.size())

        # put through each head to judge music (real / fake) and genre labels
        music_judgement = self.discriminator_music_head(common_fc_input).flatten().float()
        genre_judgement = self.discriminator_genre_head(common_fc_input).view(-1, self.n_labels).float()
        # print("dis out ", music_judgement.size(), genre_judgement.size())

        return music_judgement, genre_judgement

#### Training & Tests

In [ ]:
lpd5Train = GANTraining(MusiGen, MusiDis, lpd5.dataset)
lpd5Train.setup(20000, batch_size = 25, discriminator_rounds = 5, loss_function = "CAN")
lpd5Train.train()

In [ ]:
# storage
model_name = "MusiCAN_test1"
training_filepath = "/content/drive/MyDrive/MusiCAN_data_and_models/" + data_name + "/trained_models/"
lpd5Train.save(training_filepath, model_name)
lpd5Train.load(training_filepath, model_name)

In [ ]:
lpd5Train.plot()